Text Analysis
==
This project is a text analysis of the r/survivor subreddit's opinion on each Survivor season. The goal of this visualization is to identify main descriptors for each season. Text analysis inspired by [github/walkerkq](https://github.com/walkerkq/textmining_southpark/)

All data sourced from [r/survivor](https://www.reddit.com/r/survivor/)

In [339]:
import praw
import re
import pandas as pd
from praw.models import MoreComments
import numpy as np
from datetime import datetime, timezone
import json

# Track date of data pull
pull_date = datetime.now().strftime("%b %d %Y %H:%M:%S")

# Create connection to Reddit via PRAW
fp = open('./settings.json')
settings = json.load(fp).get('praw')
reddit = praw.Reddit(client_id = settings.get('client_id'),
                    client_secret = settings.get('client_secret'),
                    user_agent = settings.get('user_agent'),
                    username = settings.get('username'),
                    password = settings.get('password'))
subreddit = reddit.subreddit('survivor')

fp = open('./seasons.json')
seasons = json.load(fp)

processTopPosts = False

In [ ]:
# Get top 100 submissions on each season
top_submissions = {}
if(processTopPosts):
    for (nbr, season) in seasons.items():
        submissions = subreddit.search("flair:"+season.replace("\n"," "),sort='top')
        for sub in submissions:
            top_submissions[sub.id] = nbr

# Unspoiled posts
wssyw_ids = ['10tpq9', # e.g. S06: The Amazon, hyperlinked
             '26viy4', # e.g. Season 28: Cagayan
             '37c7zu', # e.g. S15: China
             '3xp433', 
             '4kzd4g', 
             '5jwzib', 
             '6ga0ty', # [Season 7: Pearl Islands](link to countdown)
             '8p0ye9', 
             'btu8iu'  # Note: Still posting countdown
            ]
wssyw_spoilers = {'1':'c48snf',
                  '2':'c0mplt',
                  '3':'c1c9cs',
                  '4':'c1r9tn',
                  '5':'bwr435',
                  '6':'c3slsk',
                  '7':'c8c3ov',
                  '8':'bx4u7z',
                  '9':'c2jn8g',
                  '10':'c24mgl',
                  '11':'c0ep8t',
                  '12':'c5xgnr',
                  '13':'c12yug',
                  '14':'by9sit',
                  '15':'c8bzdu',
                  '16':'c6b0ce',
                  '17':'c5pfk8',
                  '18':'c8c1g8',
                  '19':'bymej8',
                  '20':'c82s4d',
                  '21':'bztmw5',
                  '22':'bv9dna',
                  '23':'bzequv',
                  '24':'bwee06',
                  '25':'c6bmtt'}

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [ ]:
# Create dataframe of all comments
def get_reply_comments(submission):
    d = []
    for top_level_comment in submission.comments:
        if (not top_level_comment.is_submitter):
            continue    
        season = None
        season_name = None
        match1 = re.search("(\d+:( (\w|-)+|\.+){1,})",top_level_comment.body)
        match2 = re.search("(:(\d+)( (\w|-)+|\.+){1,})",top_level_comment.body) # Data error of S:00 Example Name
        if match1:
            season = int(match1.group(0).split(': ')[0])
            season_name = seasons[str(season)]
        if match2:
            season = int(match2.group().split(':')[1].split(' ')[0])
            season_name = seasons[str(season)]
        for comment in top_level_comment.replies.list():
            d.append({'season_nbr':season,'season':season_name,'comment_id':comment.id,'comment':comment.body,'score':comment.score})
    return d

# Get comments for each WSSYW Countdown submission
def getComments(r_id, top_level_comment=False, season_key=0):
    d = []
    submission = reddit.submission(id=r_id)
    submission.comments.replace_more(limit=2500, threshold=0)
    if top_level_comment:
        [d.append(row) for row in get_reply_comments(submission)]
    else:
        for comment in submission.comments.list():
            d.append({'season_nbr':int(season_key),'season':seasons[season_key],'comment_id':comment.id,'comment':comment.body,'score':comment.score})
    return d

In [ ]:
#### LARGE DATA SET ####
if (processTopPosts):
    d_all = []
    for (r_id,season_nbr) in top_submissions.items():
        [d_all.append(row) for row in getComments(r_id,False,season_nbr)]
    df_raw_all = pd.DataFrame(d_all)

    # Save csv for reference
    df_raw_all.to_csv('top_post_comments.csv',index = None, header=True)

#### UNSPOILED DATA ####
d=[]
for r_id in wssyw_ids:
    [d.append(row) for row in getComments(r_id, True)]

df_raw_wssyw = pd.DataFrame(d)

# Save csv for reference
df_raw_wssyw.to_csv('unspoiled_comments.csv',index = None, header=True)

In [340]:
df_raw = pd.read_csv("unspoiled_comments.csv")
df_raw_clean = df_raw[df_raw['score']>0]

# df_raw = df_raw_sp.append(df_raw_wssyw).sort_values('season_nbr').reindex()
# df_raw = df_raw_wssyw

In [341]:
from nltk.tokenize import sent_tokenize
from collections import OrderedDict

s = "'' think deserve mention breath ri caramoan ."
t = sent_tokenize(s)
seen = set()
print([len(word_tokenize(sent)) for sent in t])
print( [x for x in t if (x not in seen and not seen.add(x)) or (len(word_tokenize(x))<4)])

[8]
["'' think deserve mention breath ri caramoan ."]


In [342]:
stop = ['i', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'a', 'an', 'the', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',  'some', 'such',  's', 't', 'will', 'just', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain']
from textblob import Word
from collections import OrderedDict
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
from nltk.tokenize import sent_tokenize

fp = open('./contractions.json')
contractions = json.load(fp)
lem = WordNetLemmatizer() 

def get_wordnet_pos(sent):
    """Map POS tag to first character lemmatize() accepts"""
    sent_pos = []
    tags = pos_tag(word_tokenize(sent))
    for tag in tags:
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        sent_pos += [tag_dict.get(tag[1][0], wordnet.NOUN)]
    return sent_pos

def remove_copy_paste(x):
    sentences = sent_tokenize(x)
    seen = set()
    return " ".join([sent for sent in sentences if (sent not in seen and not seen.add(sent)) or (len(word_tokenize(sent))<8)])
def to_lower(x):
    return " ".join(x.lower() for x in x.split())
def remove_stop(x):
    return " ".join(word for word in x.split() if word not in stop)
def lem_words(x):
    contr = " ".join([contractions.get(word.replace("'",''), word.replace("'",'')) for word in x.split()])
    pos = get_wordnet_pos(contr)
    lems = []
    for w,p in zip(word_tokenize(contr), pos):
        lems+=[lem.lemmatize(w, p)]
    return " ".join(lems)

df_clean_tmp = df_raw_clean
col = df_raw_clean['comment'].apply(lambda x: to_lower(x)).apply(lambda x: lem_words(x)).str.replace('[^\w\s]|[\d]','').apply(lambda x: remove_stop(x))
df_clean_tmp = df_raw_clean.assign(comment_clean=col.values)

In [343]:
df_comb = pd.DataFrame(columns = ['comment'])

df_comb['comment'] = df_clean_tmp.groupby('season_nbr')['comment_clean'].apply(lambda x: " ".join(x))
df_comb['season_nbr'] = df_comb.index

df_clean = df_comb

In [344]:
df_comb.iloc[23,0]

'decent if like watch sound strategy play not too many shakeup though wehatecolton bad season two people even try win and one bad human ever play game best candidate fizzle end predictable no one root lot really freak stupid contestant but few memorable one all all very poor season need twist too many boring contestant not seem genuinely cruel feel like only person world think winner not mastermind think spoilerfree ranking shame fix one word so nicely not really spoiler no name mention and vague enough not make obvious refer one world one single most dominant winner all time sadly also make season really boring watch most jury either drone prop or unlikable mileage may vary furthermore another one contestant probably single most loathsome repulsive player ever take season little too personally poor quality make likely one more interesting twist idea two tribe live same beach very start probably not make another appearance least no redemption island think one well know end not expect w

In [345]:
import spacy
from spacy.lang.en import English

from nltk import ne_chunk, pos_tag, word_tokenize

def getPropNouns(data):
    chunks = ne_chunk(pos_tag(word_tokenize(data)))
    nouns = []
    for chunk in chunks:
        if hasattr(chunk, 'label'):
            if(chunk.label()=="PERSON"):
                nouns+=[chunk[0][0].lower()]
    return nouns

def getIgnoreWords(data):
    tags = pos_tag(word_tokenize(data))
    tag_dict = {
            "J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV}
    words = []
    for tag in tags:
        if (tag_dict.get(tag[1][0], wordnet.NOUN) != wordnet.NOUN):
           words+=[tag[0].lower()]
    return words
        
non_nouns = getIgnoreWords(" ".join(df_raw['comment']))

propNounsTemp = {}
df_raw['entity'] = df_raw['comment'].apply(lambda x: getPropNouns(x))
for row in df_raw['entity']:
    for item in row:
        propNounsTemp[item] = propNounsTemp.get(item, 0) + 1

In [346]:
propNouns=[]
for (key, count) in propNounsTemp.items():
    if count > 3:
        propNouns+=[key]
# print([word for word in propNouns if pos_tag([word])[0][1][0] in ['N']])
propNouns = [word for word in propNouns if word not in non_nouns]

In [347]:
rare_words = pd.Series(' '.join(df_clean['comment']).split()).value_counts()[pd.Series(' '.join(df_clean['comment']).split()).value_counts()<=1].index.values.tolist()
freq_words = pd.Series(' '.join(df_clean['comment']).split()).value_counts()[:100].index.values.tolist()

In [348]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

drop_terms = rare_words
# vec = CountVectorizer(ngram_range=(1,4), analyzer='word')
vec = TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_df=0.99)
X = vec.fit_transform(df_clean['comment'])
df_tmp = pd.DataFrame(X.toarray(), index=df_clean['season_nbr'] ,columns=vec.get_feature_names())
tdm = pd.DataFrame(df_tmp.transpose())
tdm = tdm[~tdm.index.isin(drop_terms)]
tdm.index.name = 'word'
tdm['sum'] = tdm.sum(axis=1)
tdm = tdm[~tdm.index.isin(tdm.sort_values(24.0, ascending=False)[:26][tdm[24.0]==tdm['sum']].index.tolist())]
blob = [TextBlob(word) for word in tdm.index.tolist()]
tdm['sentiment'] = [float(b.sentiment.polarity) for b in blob]
tdm['subjective'] = [float(b.sentiment.subjectivity) for b in blob]
tdm.sort_values('sum', ascending=False).head(25)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


season_nbr,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,32.0,33.0,34.0,35.0,36.0,37.0,38.0,sum,sentiment,subjective
word,,,,,,,,,,,,,,,,,,,,,
twist,0.001122,0.004797,0.038472,0.008390,0.030081,0.027065,0.140752,0.003449,0.007373,0.025389,...,0.072416,0.013661,0.090505,0.066698,0.082786,0.005497,0.126768,1.689148,0.0,0.0
favorite,0.014964,0.042657,0.062797,0.022382,0.024004,0.037686,0.033008,0.044234,0.058623,0.069464,...,0.056701,0.018688,0.023214,0.000000,0.000000,0.028201,0.005203,1.411749,0.5,1.0
winner,0.006731,0.035180,0.031477,0.005034,0.071860,0.023199,0.020107,0.006898,0.042393,0.054163,...,0.080038,0.034151,0.042020,0.015392,0.000000,0.005497,0.020283,1.370739,0.0,0.0
no,0.065070,0.015991,0.029729,0.030205,0.041779,0.007733,0.036193,0.053461,0.011059,0.020311,...,0.038114,0.047812,0.045252,0.020522,0.043572,0.000000,0.015212,1.325842,0.0,0.0
story,0.020194,0.014392,0.006995,0.057054,0.015040,0.011599,0.036193,0.025868,0.057138,0.059241,...,0.072416,0.045535,0.016162,0.010261,0.008714,0.021989,0.000000,1.225099,0.0,0.0
know,0.052729,0.020788,0.020985,0.033561,0.028410,0.019332,0.028150,0.051736,0.016589,0.022004,...,0.024774,0.018214,0.032323,0.020522,0.017429,0.000000,0.020283,1.123047,0.0,0.0
any,0.058339,0.047973,0.041970,0.033561,0.033423,0.027065,0.034853,0.044838,0.027648,0.027082,...,0.032397,0.029598,0.029091,0.005131,0.017429,0.000000,0.010141,1.090392,0.0,0.0
island,0.012662,0.006563,0.008971,0.005165,0.015431,0.027768,0.176043,0.014155,0.007564,0.012156,...,0.007821,0.004672,0.003316,0.000000,0.053645,0.011280,0.000000,1.028339,0.0,0.0
rank,0.022438,0.023986,0.024482,0.023493,0.021725,0.025132,0.025469,0.032766,0.022118,0.027082,...,0.030491,0.022768,0.006465,0.005131,0.013071,0.000000,0.005071,1.016658,-0.8,0.9


In [349]:
def df_empty(columns, dtypes, index=None):
    assert len(columns)==len(dtypes)
    df = pd.DataFrame(index=index)
    for c,d in zip(columns, dtypes):
        df[c] = pd.Series(dtype=d)
    return df
ranked = df_empty(columns = ['season','season_name','rank','word','sentiment'],dtypes=[np.float64,np.str,np.float64,np.str,np.str])
for col in tdm.drop(columns=['sentiment','subjective','sum']).columns:
    df_new = df_empty(columns = ['season','season_name','rank','word','sentiment'],dtypes=[np.float64,np.str,np.float64,np.str,np.str])
    df_new['word'] = tdm.sort_values(col, ascending=False).index.tolist()
    df_new['rank'] = df_new.index
    df_new['rank'] = pd.to_numeric(df_new['rank'])
    df_new['season'] = float(col)
    df_new['season_name'] = seasons.get(str(int(col)))
    df_new['sentiment'] = tdm.sort_values(col, ascending=False)['sentiment'].apply(lambda x: 'very positive' if x >= 0.5 else 'positive' if 0.5 > x > 0.15 else 'very negative' if x <= -0.5 else 'negative' if -0.5 < x < -0.05 else 'neutral').tolist()
    ranked = pd.concat([ranked,df_new[:26]])
ranked

,season,season_name,rank,word,sentiment
0,1.0,Borneo,0.0,borneo,neutral
1,1.0,Borneo,1.0,different,neutral
2,1.0,Borneo,2.0,should,neutral
3,1.0,Borneo,3.0,order,neutral
4,1.0,Borneo,4.0,no,neutral
...,...,...,...,...,...
21,38.0,Edge of\nExtinction,21.0,returnees,neutral
22,38.0,Edge of\nExtinction,22.0,idol,neutral
23,38.0,Edge of\nExtinction,23.0,low ranking,neutral
24,38.0,Edge of\nExtinction,24.0,survivor become,neutral


In [350]:
from plotnine import *

# Create graphic
# Define colors
color_background = "white"
color_text = "#22211d"

my_theme = (

    # Begin construction of chart
    theme_bw(base_size=15) +

    # Format background colors
    theme(panel_background = element_rect(fill=color_background, color=color_background)) +
    theme(plot_background  = element_rect(fill=color_background, color=color_background)) +
    theme(panel_border     = element_rect(color=color_background)) +
    theme(strip_background = element_rect(fill=color_background, color=color_background)) +

    # Format the grid
    theme(panel_grid_major_y = element_blank()) +
    theme(panel_grid_minor_y = element_blank()) +
    theme(axis_ticks         = element_blank()) +

    # Format the legend
    theme(legend_position = "none") +

    # Format title and axis labels
    theme(plot_title       = element_text(color=color_text, size=20, weight = "bold")) +
    theme(axis_title_x     = element_text(size=10, color="black", weight = "bold")) +
    theme(axis_title_y     = element_text(size=10, color="black", weight = "bold")) +
    theme(axis_text_x      = element_text(size=10, vjust=0.5, hjust=0.5, color = color_text)) +
    theme(axis_text_y      = element_text(size=12, color = color_text)) +
    theme(strip_text       = element_text(face = "bold")) +

    # Plot margins
    theme(figure_size = (18,6))
    )

cmap_era = {"1.0":"#F70020",
            "2.0":"#1A7D00",
            "3.0":"#0C96F2",
            "4.0":"#FB9701",
            "5.0":"#636666",
            "6.0":"#87603E",
            "7.0":"#BFBEBB",
            "8.0":"#FB9701",
            "9.0":"#636666",
            "10.0":"#87603E"
           }
cmap_tone = {"very positive":"#4CAF50","positive":"#8BC34A","neutral":"#B3B6B7","negative":"#FFC107","very negative":"#F4511E"}

In [351]:
def base_fig(data, fill_col):
    season_names = pd.Categorical(data['season_name'], categories=data['season_name'].unique().tolist())
    data = data.assign(season_name_ord = season_names)
    return(
        ggplot(data=data, mapping=aes(x='season_name_ord', y='rank')) +
        geom_point(color="black") + 
        geom_label(data=data, 
                   mapping=aes(label='word', fill=fill_col), 
                   color='white',
                   size=10,
                   label_padding=0.15) +
        geom_label(data=data.loc[data['word'].isin(propNouns)], 
           mapping=aes(label='word'), 
           fill='white',
           color='grey',
           size=10,
           label_padding=0.15) +
        scale_y_reverse(limits=[1,25], breaks=range(25,0,-1)) +
        labs(x="Season", y="Ranking",title="Most Characteristic Words Used to Describe a Season") +
        scale_fill_manual(cmap_tone)
    )

In [352]:
first_10 = ranked[ranked['season']<11]
fig1 = (base_fig(first_10, 'sentiment') + my_theme)
ggsave(plot=fig1, filename='seasons1to10.png')

/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 18 x 6 in image.
  from_inches(height, units), units), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: seasons1to10.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_point : Removed 10 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_label : Removed 10 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_label : Removed 3 rows containing missing values.
  self.data = self.geom.handle_na(self.data)


In [353]:
second_10 = ranked[(ranked['season']>10) & (ranked['season']<21)]
fig2 = (base_fig(second_10, 'sentiment') + 
        my_theme  
       )
ggsave(plot=fig2, filename='seasons11to20.png')

/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 18 x 6 in image.
  from_inches(height, units), units), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: seasons11to20.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_point : Removed 10 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_label : Removed 10 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_label : Removed 3 rows containing missing values.
  self.data = self.geom.handle_na(self.data)


In [354]:
third_10 = ranked[(ranked['season']>20) & (ranked['season']<31)]
fig3 = (base_fig(third_10, 'sentiment') + my_theme )
ggsave(plot=fig3, filename='seasons21to30.png')

/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 18 x 6 in image.
  from_inches(height, units), units), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: seasons21to30.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_point : Removed 10 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_label : Removed 10 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_label : Removed 1 rows containing missing values.
  self.data = self.geom.handle_na(self.data)


In [356]:
fourth_10 = ranked[(ranked['season']>30)]
fig4 = (base_fig(fourth_10, 'sentiment') + my_theme )
ggsave(plot=fig4, filename='seasons31to38.png')

/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 18 x 6 in image.
  from_inches(height, units), units), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: seasons31to38.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_point : Removed 8 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
/anaconda3/lib/python3.7/site-packages/plotnine/layer.py:452: PlotnineWarning: geom_label : Removed 8 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
